# 안전 벡터맨

## 건설 현장 안전 사고방지 AI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade jsonlines openai langchain langchain-openai langchain-community -q
!pip install chromadb==0.5.3 langchain-chroma tiktoken rank_bm25 -q
!pip install pymupdf pypdf pypdf2 -q

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "your_openai_api_key"
# 환경변수에 OPENAI_API_KEY를 설정합니다.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
from langchain.schema import Document
from glob import glob

path = '/content/drive/MyDrive/skala/건설현장지침/*.pdf' # './drive/MyDrive/8_papers/*.pdf'

glob(path)

In [ ]:
import glob
import re
from langchain_community.document_loaders import PyPDFLoader  # PyPDFLoader 추가
from langchain.schema import Document

# PDF 파일 경로 설정
pdf_files = glob.glob('/content/drive/MyDrive/skala/건설현장지침/*.pdf')

# 전처리 함수 정의
def clean_text(text):
    # 페이지 번호 및 특수문자 제거
    text = re.sub(r'\n{2,}', ' ', text)  # 연속 개행을 공백으로 변환
    text = re.sub(r'[\n]', ' ', text)     # 일반 개행도 공백으로 변환
    text = re.sub(r'[^\S\n]+', ' ', text) # 다중 공백 제거
    text = text.strip()  # 양끝 공백 제거
    return text

# PDF 파일 로드 및 전처리
all_papers = []

for i, path_paper in enumerate(pdf_files):
    try:
        loader = PyPDFLoader(path_paper)  # PyPDFLoader 사용
        pages = loader.load()
    except:
        print(f"{path_paper} 로드 실패, 스킵합니다.")
        continue

    # PDF 페이지 병합 및 전처리
    doc = Document(page_content='', metadata={'index': i, 'source': pages[0].metadata['source']})
    for page in pages:
        clean_content = clean_text(page.page_content)
        doc.page_content += clean_content + " "

    all_papers.append(doc)

print(f"총 {len(all_papers)}개의 문서가 로드되었습니다.")
print("샘플 문서:", all_papers[:3])


In [ ]:
import tiktoken

encoder = tiktoken.encoding_for_model('gpt-4o-mini') # 텍스트를 토큰으로 변환하기 위해, OpenAI 제공 패키지 tiktoken 사용
for paper in all_papers:
    print(len(encoder.encode(paper.page_content)), paper.metadata['source']) # 각 문서별 토큰 갯수 확인

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken
token_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",
    chunk_size=800, # 800 토큰 단위 (GPT-4o-mini 기준)
    chunk_overlap=80,
)


token_chunks = token_splitter.split_documents(all_papers)
print(len(token_chunks))

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small') # 'text-embedding-3-large'

Chroma().delete_collection()
db = Chroma.from_documents(documents=token_chunks, # 이 코드에서 Chroma.from_document()이 실행되면 embedding을 사용하여 텍스트를 벡터로 변환하고 DB에 저장함.
                           embedding=embeddings,
                           #persist_directory="./chroma_Web", #  디스크에 저장하고 싶을 때 사용 #'./drive/MyDrive/8_papers./*.pdf'
                           collection_metadata={'hnsw:space':'l2'} # ChromaDB는 고차원 벡터 검색을 빠르게 수행하기 위해 HNSW 알고리즘을 사용, "hnsw:space"는 HNSW 검색 공간 거리 계산 방식 지정 키, "l2"는 L2 거리(유클리드 거리, Euclidean Distance) 의미
                           )
# Top 5 검색하기
retriever = db.as_retriever(search_kwargs={"k": 5}) # db에서 벡터 검색을 수행할 수 있도록 설정함, 데이터 저장이 완료된 이후에 retriever를 통해 검색 가능

# filter 옵션을 통해 특정 메타데이터를 가진 벡터만 검색 가능
# retriever = db.as_retriever(search_kwargs={"k": 5,"filter":{'author':'Sugnryel Lim'}})

retriever.invoke('안전')

In [ ]:
# vector store에서 유사도 확인하기
query = "How does Exaone achieve good evaluation results?"
db.similarity_search_with_score(query) # 유사도 스코어는 0 ~ 1 사이 값을 가지며, 1에 가까울 수록 더 유사한 문서이다.

# 참고로, HNSW에서 l1은 코사인 유사도로 -1 ~ 1 사이 값을 가진다. 두 벡터가 동일한 방향을 가리킬 수록 1에 가까워지며 유사도가 높음
  # cos 0∘ = 1 → 완전히 같은 방향 (유사도가 1)
  # cos 90∘ = 0 → 서로 직교 (연관성이 없음)
  # cos 180∘ = −1 → 완전히 반대 방향 (유사도가 -1)

# l2는 유클리드 거리 유사도로, 0에 가까울 수록 유사도가 높다. (벡터들이 더 가까움)
  # 두 점(벡터) 사이의 직선 거리(즉, 피타고라스 정리 기반의 거리)를 측정하여 유사도를 판단하는 방식
  # 두 벡터의 차이의 제곱합을 계산하여 루트를 씌운 값이 유클리드 거리

In [ ]:
def retriever_with_score(query):
    docs, scores = zip(*db.similarity_search_with_score(query)) # * 연산자는 리스트의 각 요소를 개별적으로 풀어서(zip 해체) 따로 추출함, 여기서는 scores만 따로 추출
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score

    return docs

In [ ]:
# Query 검색
# RunnableLambda : 함수를 Runnable로 Wrap

from langchain_core.runnables import RunnableLambda
unique_docs = RunnableLambda(
    retriever_with_score).invoke("How does Exaone achieve good evaluation results?")
# 함수에 직접 invoke를 실행 가능하도록 RunnableLamda()로 묶음
unique_docs

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)


In [ ]:
# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("user", '''당신은 건설 현장 안전 도우미 AI입니다. 다음의 Context를 이용하여 Question에 답변하세요.

### Chain-of-Thought : 안전 지침을 열고 해당 작업에 대한 준비 사항을 확인한다. 준비 사항 중 잊지 말아야 할 것을 기록한다.
시공 순서를 탐색한다. 예를 들어 지하 연속벽의 경우 안내벽 설치부터 콘크리트 타설까지 진행된다. 공정 순서를 우선 나열한 뒤, 각 공정에 대해 주요 안전사항을 기록한다.
안전화나 고도와 같은 기초 안전 내용보다는 압송 장치와 같이 디테일해서 놓칠 수 있는 부분을 찾아서 기록한다.
만일 작업의 특정 공정에 대해 물어본 경우, 내용을 구체적으로 기록한다.

### Condition :
1. 공정 순서를 먼저 나열한 후, 각 공정 단계별로 소제목을 구성하고, 단계별 중요도가 높은 사항을 기준으로 작성한다.
2. 어느 조항을 참조하는 지 작성하지 않는다.
3. 만약 모든 Context를 다 확인해도 정보가 없다면, 웹 검색을 통해 추가 정보를 검색한다.
4. 답변은 한국어로 작성한다.
5. 모든 시공 절차에 대한 내용을 작성한다.

### Output Example :
공정 순서:
1. 안내벽 설치 단계
2. 플랜트 설치 단계
3. 선행굴착 및 본굴착 단계

1. 안내벽 설치 단계
   a. 안내벽 설치 전, 지반 붕괴 위험이 있으면 흙막이지보공을 설치한다.
   b. 야간작업 시 충분한 조명(75럭스 이상)과 형광벨트, 경광등을 설치한다.
   c. 굴착장비가 사면에 지나치게 근접해 붕괴되지 않도록 한다.

2. 플랜트 설치 단계
   a. 구동벨트는 철망으로 감싸 끼임 사고를 방지한다.
   b. 사일로 점검을 위한 사다리 및 난간을 반드시 설치한다.
   c. 배관 연결 상태를 점검하고, 펌프 압력으로 유동되지 않도록 고정한다.

3. 선행굴착 및 본굴착 단계
   a. 트렌치 커터 이동 시 지반침하 방지를 위해 콘크리트를 타설하거나 철판을 깐다.
   b. 크레인 작업 구역에 근로자가 접근하지 않도록 통제한다.
   c. 굴착 중 지하 매설물(가스관, 상수도관 등) 위치를 사전에 확인하고 보호 조치를 취한다.
---
Context: {context}
---
Question: {question}''')
])

prompt.pretty_print()



In [ ]:
translate_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '주어진 질문을 영어로 변환하세요.'),
        ('user', 'Question: {question}')
    ]
)
translate_chain = translate_prompt | llm | StrOutputParser()

def format_docs(docs):
    return "\n---\n".join([doc.page_content+ '\nURL: '+ doc.metadata['source'] for doc in docs])
    # join : 구분자를 기준으로 스트링 리스트를 하나의 스트링으로 연결

rag_chain = (
    {"context": translate_chain | retriever | format_docs, "question": RunnablePassthrough()}
    # context는 질문을 번역하고 검색한 문서를 텍스트로 변환한 것이며, question은 원래 질문을 그대로 유지
    # retriever : question을 받아서 context 검색: document 반환, format_docs : document 형태를 받아서 텍스트로 변환
    # RunnablePassthrough(): 체인의 입력을 그대로 저장
    | prompt
    # context (검색된 문서)와 question (질문)을 이용하여 LLM에게 입력할 최종 프롬프트를 생성
    | llm
    | StrOutputParser()
)

In [ ]:
questions = [
    '오늘  SCW 공법을 사용해서 흙막이 공사 할건데 안전 수칙 알려줘'
]
result = rag_chain.batch(questions)
for i, ans in enumerate(result):
    ans = ans.replace('.','.\n')
    print(f"Question: {questions[i]}")
    print(f"Answer: {ans}")
    print('---')


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import asyncio

def analyze_excavation_safety(question):
    """
    질문을 받아 RAG 모델을 호출하여 안전 정보를 제공하는 함수
    """
    result = rag_chain.invoke(question)  # RAG 체인을 활용한 응답 생성
    formatted_result = result.replace('.', '.\n')  # 가독성 향상
    return formatted_result

# ✅ Gradio UI 구성
with gr.Blocks(css=".gradio-container {max-width: 800px; margin: auto; font-family: 'Arial', sans-serif;}") as demo:
    gr.Markdown("## 🏗️ 건설현장 안전사고 방지 AI")
    gr.Markdown("건설현장에서 발생할 수 있는 사고에 대해 질문하면, 벡터 DB와 RAG 기술을 활용한 학습 데이터 기반의 정보를 제공합니다.")

    with gr.Row():
        with gr.Column():
            question_input = gr.Textbox(
                label="💬 건설현장 관련 사고 질문 입력",
                placeholder="예: 오늘 SCW 공법을 사용할 건데 안전 수칙 알려줘",
                lines=2,
                interactive=True
            )
            submit_button = gr.Button("🚀 분석 실행")

        with gr.Column():
            output_box = gr.Textbox(
                label="📝 분석 결과",
                placeholder="AI가 분석한 결과가 여기에 표시됩니다.",
                lines=10,
                interactive=False
            )

    submit_button.click(fn=analyze_excavation_safety, inputs=[question_input], outputs=output_box)

    gr.Markdown("---")
    gr.Markdown("&copy; 2025 안전 벡터맨 All rights reserved")

# 실행
demo.launch(share=True)